In [1]:
from os import listdir
from pickle import dump
import tensorflow as tf
import pandas as pd
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
import cv2
import numpy as np
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tqdm import tqdm
#R_LOSS_FACTOR = 10000
#import numpy as np

In [3]:
RUN_FOLDER='/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/'
new_model = load_model("/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/val_acc_9280_model.h5")

In [4]:
learning_rate=K.eval(new_model.optimizer.lr)
learning_rate

1.6e-07

In [10]:
image_dataframe=pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/image_location_venky.csv')
cengage= pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/cengage_production_data.csv')
pearson= pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/pearson_production_data.csv')
wiley=pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/wiley_production_data.csv')
wiley.rename(columns={'image_names':'Name'}, inplace=True)
pearson.rename(columns={'image_names':'Name'}, inplace=True)
cengage.rename(columns={'image_names':'Name'}, inplace=True)

image_dataframe=pd.merge(image_dataframe, cengage, on='Name', how='left')
image_dataframe.drop(['image_path', 'alt_text_html', 'Unnamed: 0'], axis=1, inplace=True)

image_dataframe=pd.merge(image_dataframe, pearson, on='Name', how='left')
image_dataframe["alt_text"] = image_dataframe["alt_text_x"].astype(str) + " "+image_dataframe["alt_text_y"].astype(str)
image_dataframe['subject_area'] = image_dataframe['subject_area_x'].astype(str) + " "+ image_dataframe['subject_area_y'].astype(str)
image_dataframe.drop(['alt_text_x', "alt_text_y", 'subject_area_x', 'subject_area_y', 'alt_text_html', 'image_path'], axis=1, inplace=True)

image_dataframe=pd.merge(image_dataframe, wiley, on='Name', how='left')
image_dataframe["alt_text"] = image_dataframe["alt_text_x"].astype(str) + " "+ image_dataframe["alt_text_y"].astype(str)
image_dataframe['subject_area'] = image_dataframe['subject_area_x'].astype(str) + " "+ image_dataframe['subject_area_y'].astype(str)
image_dataframe.drop(['alt_text_x', "alt_text_y", 'subject_area_x', 'subject_area_y', 'alt_text_html', 'image_path'], axis=1, inplace=True)

banned = ['nan']
f = lambda x: ' '.join([item for item in x.split() if item not in banned])
image_dataframe["subject_area"] = image_dataframe["subject_area"].apply(f)
image_dataframe["alt_text"] = image_dataframe["alt_text"].apply(f)
image_dataframe = image_dataframe[image_dataframe.subject_area != '']
image_dataframe.subject_area=image_dataframe.subject_area.replace({"other other": "others", "other": "others"})
image_dataframe.reset_index(drop=True, inplace=True)

include = ['maths', 'science', 'others', 'emss', 'canada', 'engineering', 'business', 'history']
indexNames=[]
for i in range(image_dataframe.shape[0]):
    if(image_dataframe.loc[i]['subject_area'] not in include):
        indexNames.append(i)
image_dataframe.drop(labels=indexNames, inplace=True)
image_dataframe.reset_index(drop=True, inplace=True)

In [11]:
image_dataframe['subject_area'].value_counts()

maths          146839
science         10820
others           5116
emss             3602
canada           2852
engineering      1854
business         1681
history          1014
Name: subject_area, dtype: int64

In [12]:
image_dataframe.subject_area.unique()

array(['maths', 'history', 'science', 'engineering', 'business', 'others',
       'emss', 'canada'], dtype=object)

In [13]:
image_dataframe.isna().sum()

Name            0
Path            0
Height          0
Width           0
alt_text        0
subject_area    0
dtype: int64

In [14]:
image_dataframe.shape

(173778, 6)

In [15]:
image_dataframe.head()

,Name,Path,Height,Width,alt_text,subject_area
0,9781337111348_11348_ch02_2.1_017-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,131,181,The image consists of a Minitab output. Visual...,maths
1,9781285194790_9781285194790-551-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,550,593,A map shows the Second Iraq War. Majority popu...,history
2,9781133947257_9781133947257_ch01_65-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,101,184,In the Sherlock Holmes mystery The Final Solut...,maths
3,9781305652231_52231_ch01_f022-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,24,188,The image consists of a number line. The numbe...,maths
4,ceng_image205.png,/home/pytorch_ashish/deep_learning/data/alt_te...,131,268,The image consists of a visual representation ...,maths


In [11]:
def gray(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = np.expand_dims(gray_image, axis=-1)
    return gray_image
BATCH_SIZE=32
trdata = ImageDataGenerator(rescale=1./255,  preprocessing_function=gray)
traindata = trdata.flow_from_dataframe(dataframe = image_dataframe
                                         , directory = None
                                         , x_col='Path'
                                         , y_col='subject_area'
                                         , target_size = (224,224)
                                         , batch_size = BATCH_SIZE
                                         , shuffle = False
                                         #, classes = y_train_en
                                         , class_mode = 'categorical'
                                            )


Found 173778 validated image filenames belonging to 8 classes.


In [12]:
int(round(image_dataframe.shape[0]/BATCH_SIZE ,0))

5431

In [13]:
image_dataframe.shape[0]/32

5430.5625

In [19]:
def extract_features_gen(dataframe):
   
    final_model = Model(inputs=new_model.inputs, outputs=new_model.layers[-2].output)
    # summarize
    print(final_model.summary())
   
    features = []
    for i in tqdm(range(int(round(dataframe.shape[0]/BATCH_SIZE ,0)))):
    
        images, labels = next(traindata)
        # get features
        feature = final_model.predict(images, verbose=0)
        
        features.extend(feature)
        
    features=np.array(features)
    return features

In [20]:
from tqdm import tqdm
features = extract_features_gen(image_dataframe)
print('Extracted Features: %d' % len(features))
RUN_FOLDER='/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/'
# save to file
dump(features, open(RUN_FOLDER+'features.pkl', 'wb'))

0%|          | 0/5431 [00:00<?, ?it/s]Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooli

In [21]:
features.shape

(173778, 4096)

In [9]:
def clean_sentence(sent):
	
    sent = sent.lower()
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    sent =sent.replace(' x ', ' x-axis ')
    sent=sent.replace(' y ', ' y-axis ')
    word_tokens = word_tokenize(sent)
    sent = ' '.join(w for w in word_tokens if not w in stop_words and len(w)>2)
    sent= re.sub(r"(\d+\s?)+"," number ",sent)

    return sent


In [10]:
for i in tqdm(range(image_dataframe.shape[0])):
    clean=clean_sentence(image_dataframe.loc[i]['alt_text'])
    #print(clean)
    image_dataframe['alt_text'][i]='startseq ' + clean + ' endseq'

100%|██████████| 173778/173778 [06:40<00:00, 433.37it/s]


In [13]:
#image_dataframe['alt_text'][0]=clean
image_dataframe['alt_text'][12456]

'startseq four curves graphed x-axis y-axis coordinate plane horizontal axis labeled x-axis vertical axis labeled y-axis curves labeled y-axis x-axis x number c number value different curve curve enters bottom third curve goes right enters second quadrant intersects y-axis axis enters first quadrant curve enters bottom third curve goes right enters second quadrant intersects y-axis axis enters first quadrant curve enters bottom third curve goes right passes origin enters first quadrant goes right negative curve enters bottom third curve goes right intersects negative y-axis axis negative goes right enters first quadrant curve negative curve curve curve endseq'

In [14]:
image_dataframe.shape

(173778, 6)

In [17]:
image_dataframe.to_csv('cleantext_alt_text.csv', index=None)

In [3]:
test= pd.read_csv('cleantext_alt_text.csv')
test.head()

,Name,Path,Height,Width,alt_text,subject_area
0,9781337111348_11348_ch02_2.1_017-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,131,181,startseq image consists minitab output visual ...,maths
1,9781285194790_9781285194790-551-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,550,593,startseq map shows second iraq war majority po...,history
2,9781133947257_9781133947257_ch01_65-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,101,184,startseq sherlock holmes mystery final solutio...,maths
3,9781305652231_52231_ch01_f022-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,24,188,startseq image consists number line numbers nu...,maths
4,ceng_image205.png,/home/pytorch_ashish/deep_learning/data/alt_te...,131,268,startseq image consists visual representation ...,maths


In [4]:
test.shape

(173778, 6)

In [5]:
dict={}
def vocab(sent):
    word_tokens = word_tokenize(sent)
    for i in word_tokens:
        try:
            dict[i]+=1
        except:
            dict[i]=1

In [6]:
for i in tqdm(range(test.shape[0])):
    vocab(test.loc[i]['alt_text'])

100%|██████████| 173778/173778 [01:30<00:00, 1918.80it/s]


In [7]:
len(dict)

74489

In [9]:
vocabulary= set(dict.keys())

In [10]:
len(vocabulary)

74489

In [11]:
test.shape

(173778, 6)

In [12]:
X= test.loc[:,:]
y= test.loc[:,'subject_area']
X_train=X[0:130333]
X_test=X[130333:]
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

In [13]:
X_train.shape

(130333, 6)

In [14]:
X_test.shape

(43445, 6)

In [15]:
all_alt_text = list()
for i in tqdm(range(X_train.shape[0])):
    all_alt_text.append(X_train.loc[i]['alt_text'])


100%|██████████| 130333/130333 [00:28<00:00, 4565.71it/s]


In [16]:
test_all_alt_text = list()
for i in tqdm(range(X_test.shape[0])):
    test_all_alt_text.append(X_test.loc[i]['alt_text'])


100%|██████████| 43445/43445 [00:09<00:00, 4610.37it/s]


In [17]:
print(len(all_alt_text), len(test_all_alt_text))

130333 43445


In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_alt_text)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

52413

In [19]:
import pickle
filename='features.pkl'
infile = open(filename,'rb')
photo_features = pickle.load(infile)
infile.close()

In [20]:
photo_features.shape

(173778, 4096)

In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from numpy import array

In [22]:
import threading

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return self.it.__next__()


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g


In [23]:
@threadsafe_generator
def data_generator(batch_size,tokenizer, vocab_size, photo_features, all_alt_text):
    global generator_index
    print("Generator Initiated", generator_index)
    generator_index+=1
    while True:		
        num_samples = len(all_alt_text)
        for offset in range(0, num_samples, batch_size):
            photo = photo_features[offset : offset + batch_size]
            alt_text = all_alt_text[offset : offset + batch_size]
            print(" ",offset, " I am offset of Generator index ",generator_index)
            
            X1, X2, y = list(), list(), list()
            for j in range(len(alt_text)):
                seq = tokenizer.texts_to_sequences([alt_text[j]])[0]
                
                for i in range(1, len(seq)):
                    in_seq, out_seq =seq[:i], seq[i]
                    
                    in_seq = pad_sequences([in_seq], maxlen=200)[0]   #200 = max_length
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    
                    X1.append(photo[j])
                    X2.append(in_seq)
                    y.append(out_seq)
                    
                    
            #in_img, in_seq, out_word = create_sequences(tokenizer, vocab_size, photo, alt_text)
            yield [array(X1), array(X2)], array(y)
            

In [51]:
from tensorflow.keras.utils import Sequence
class Data_Gen(Sequence):
    '''Generates Data for Keras'''
    def __init__(self, batch_size, tokenizer, vocab_size, photo_features, alt_text):
        '''Initialization'''
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.vocab_size = vocab_size
        self.photo_features = photo_features
        self.alt_text = alt_text
        self.on_epoch_end()
    
    def __len__(self):           #Must be implemented for every Sequence Item
        '''Denotes the no of batches per epoch'''
        return int(np.floor(self.photo_features.shape[0]/self.batch_size))
    
    def __getitem__(self, index):       #Must be implemented for every Sequence Item and it should return a complete Batch
        '''Generate one batch of data'''
        # Generate indexes of the batch
        #indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]   #indexes is not present in the Main Sequence Keras Class
        photo = self.photo_features[index*self.batch_size:(index+1)*self.batch_size]
        alt_text = self.alt_text[index*self.batch_size:(index+1)*self.batch_size]
        
        X1, X2, y = list(), list(), list()
        for j in range(len(alt_text)):
            seq = tokenizer.texts_to_sequences([alt_text[j]])[0]

            for i in range(1, len(seq)):
                in_seq, out_seq =seq[:i], seq[i]

                in_seq = pad_sequences([in_seq], maxlen=200)[0]   #200 = max_length
                # encode output sequence
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                X1.append(photo[j])
                X2.append(in_seq)
                y.append(out_seq)


        #in_img, in_seq, out_word = create_sequences(tokenizer, vocab_size, photo, alt_text)
        return [array(X1), array(X2)], array(y)
    
    def __iter__(self):
        """Create a generator that iterate over the Sequence."""
        for item in (self[i] for i in range(len(self))):
            yield item

    
    #def on_epoch_end(self):                           Optional function can be used to do shuffling
     #   'Updates indexes after each epoch'
      #  self.indexes = np.arange(len(self.list_IDs))
       # if self.shuffle == True:
        #    np.random.shuffle(self.indexes)
        

In [34]:
train_photo_features=photo_features[0:130333]
test_photo_features=photo_features[130333:]
print(train_photo_features.shape, test_photo_features.shape)

(130333, 4096) (43445, 4096)


In [ ]:
index= [0]

In [38]:
print(int(np.floor(train_photo_features.shape[0]/32)))
np.arange(len(self.list_IDs))

4072

In [60]:
global generator_index
generator_index =-1

In [52]:
batch_size=3
generator = Data_Gen(batch_size, tokenizer, vocab_size, train_photo_features, all_alt_text)
inputs, outputs = next(generator)
print(inputs[0].shape)
print(inputs[1].shape)
print(outputs.shape)

TypeError: 'Data_Gen' object is not an iterator

In [33]:
#global test_generator_index
#test_generator_index=0
batch_size=3
#test_generator = data_generator(batch_size, tokenizer, vocab_size, test_photo_features, test_all_alt_text)
test_inputs, test_outputs = next(test_generator)
print(test_inputs[0].shape)
print(test_inputs[1].shape)
print(test_outputs.shape)

(89, 4096)
(89, 200)
(89, 52413)


In [173]:
#print(len(tokenizer.texts_to_sequences([all_alt_text[0]])[0])-1)
#print(len(tokenizer.texts_to_sequences([all_alt_text[0]])[0])+len(tokenizer.texts_to_sequences([all_alt_text[1]])[0])-2)
print(len(tokenizer.texts_to_sequences([all_alt_text[0]])[0])+len(tokenizer.texts_to_sequences([all_alt_text[1]])[0])+len(tokenizer.texts_to_sequences([all_alt_text[2]])[0]) -3)
print(len(tokenizer.texts_to_sequences([all_alt_text[3]])[0])+len(tokenizer.texts_to_sequences([all_alt_text[4]])[0])+len(tokenizer.texts_to_sequences([all_alt_text[5]])[0]) -3)
print(len(tokenizer.texts_to_sequences([all_alt_text[6]])[0])+len(tokenizer.texts_to_sequences([all_alt_text[7]])[0])+len(tokenizer.texts_to_sequences([all_alt_text[8]])[0]) -3)
print(len(tokenizer.texts_to_sequences([all_alt_text[9]])[0])+len(tokenizer.texts_to_sequences([all_alt_text[10]])[0])+len(tokenizer.texts_to_sequences([all_alt_text[11]])[0]) -3)

22
108
120
150
283
96


In [41]:
#print(len(tokenizer.texts_to_sequences([test_all_alt_text[0]])[0])-1)
#print(len(tokenizer.texts_to_sequences([test_all_alt_text[0]])[0])+len(tokenizer.texts_to_sequences([test_all_alt_text[1]])[0])-2)
print(len(tokenizer.texts_to_sequences([test_all_alt_text[0]])[0])+len(tokenizer.texts_to_sequences([test_all_alt_text[1]])[0])+len(tokenizer.texts_to_sequences([test_all_alt_text[2]])[0]) -3)
print(len(tokenizer.texts_to_sequences([test_all_alt_text[3]])[0])+len(tokenizer.texts_to_sequences([test_all_alt_text[4]])[0])+len(tokenizer.texts_to_sequences([test_all_alt_text[5]])[0]) -3)
print(len(tokenizer.texts_to_sequences([test_all_alt_text[6]])[0])+len(tokenizer.texts_to_sequences([test_all_alt_text[7]])[0])+len(tokenizer.texts_to_sequences([test_all_alt_text[8]])[0]) -3)
print(len(tokenizer.texts_to_sequences([test_all_alt_text[9]])[0])+len(tokenizer.texts_to_sequences([test_all_alt_text[10]])[0])+len(tokenizer.texts_to_sequences([test_all_alt_text[11]])[0]) -3)

37
56
74
104
80
89


In [42]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import add
from tensorflow.keras.callbacks import ModelCheckpoint

def define_model(vocab_size, max_length):
	# feature extractor model
	inputs1 = Input(shape=(4096,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(25, activation='relu')(fe1)       #256
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = LSTM(25)(se2)                           #256
	# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(25, activation='relu')(decoder1)           #256
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	# compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	# summarize model
	model.summary()
	plot_model(model, to_file='model.png', show_shapes=True)
	return model

In [43]:
max_length=200
vocab_size= 52413
model = define_model(vocab_size, max_length)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 4096)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 256)     13417728    input_6[0][0]                    
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 4096)         0           input_5[0][0]                    
____________________________________________________________________________________________

In [44]:
print(batch_size, vocab_size, max_length, train_photo_features.shape, len(all_alt_text))

3 52413 200 (130333, 4096) 300


In [45]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TerminateOnNaN, TensorBoard, EarlyStopping
import os
from datetime import datetime
RUN_FOLDER='/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/'
logdir = os.path.join(RUN_FOLDER, "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=logdir)

#checkpoint_filepath=os.path.join(run_folder, "model.h5")
checkpoint1 = ModelCheckpoint(os.path.join(RUN_FOLDER, "caption_model.h5"), monitor='loss',  save_best_only=True, verbose=1, mode= 'min')
checkpoint2 = ModelCheckpoint(os.path.join(RUN_FOLDER, 'caption_weights.h5'), save_weights_only = True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=1, mode='min', verbose=1)
EarlyStop = EarlyStopping(monitor='loss', min_delta=0.001, patience=31, verbose=1, mode='min', restore_best_weights=True)
terminate = TerminateOnNaN()


callbacks_list = [checkpoint1, checkpoint2, reduce_lr, tensorboard_callback, EarlyStop, terminate]

In [44]:
NUM_IMAGES = train_photo_features.shape[0]
print(NUM_IMAGES)
TEST_NUM_IMAGES = test_photo_features.shape[0]
print(TEST_NUM_IMAGES)

130333
43445


In [41]:
model.fit([inputs[0], inputs[1]], outputs, epochs=1, verbose=2, validation_data=([test_inputs[0], test_inputs[1]], test_outputs))

Train on 96 samples, validate on 89 samples
96/96 - 5s - loss: 10.7925 - val_loss: 10.3645


In [53]:
batch_size=16
train_photo_features=train_photo_features[0:300]
NUM_IMAGES = train_photo_features.shape[0]
#print(NUM_IMAGES)
all_alt_text= all_alt_text[0:300]
#print(len(all_alt_text))
generator = data_generator(batch_size, tokenizer, vocab_size, train_photo_features, all_alt_text)

test_photo_features=test_photo_features[0:300]
TEST_NUM_IMAGES = test_photo_features.shape[0]
#print(TEST_NUM_IMAGES)
test_all_alt_text=test_all_alt_text[0:300]
#print(len(test_all_alt_text))
batch_size=16
test_generator = data_generator(batch_size, tokenizer, vocab_size, test_photo_features, test_all_alt_text)

global generator_index
generator_index =-1

In [ ]:
model.fit(
    x = generator
    , shuffle = False
    , epochs = 100
    , initial_epoch = 0
    , callbacks = callbacks_list
    , steps_per_epoch = NUM_IMAGES / batch_size
    , workers = 1
    , use_multiprocessing = True
    #, validation_data= test_generator
    #, validation_steps= TEST_NUM_IMAGES / batch_size
)

  16  I am offset of Generator index  0
  ...
    to  
  ['...']
Train for 18.75 steps
Epoch 1/100


Exception in thread Thread-40:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 970, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 962, in pool_fn
    initargs=(seqs, self.random_seed, get_worker_id_queue()))
  File "/opt/conda/lib/python3.6/multiprocessing/context.py", line 119, in Pool
    context=self.get_context())
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 174, in __init__
    self._repopulate_pool()
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 239, in _repopulate_pool
    w.start()
  File "/opt/conda/lib/python3.6/multiproc

In [31]:
model.fit(
    x = generator
    , shuffle = False
    , epochs = 100
    , initial_epoch = 0
    , callbacks = callbacks_list
    , steps_per_epoch = NUM_IMAGES / batch_size
    , workers = 3
    , use_multiprocessing = False
    #, validation_data= test_generator
    #, validation_steps= TEST_NUM_IMAGES / batch_size
)

Generator Initiated -1
  0  I am offset of Generator index  0
  ...
    to  
  ['...']
Train for 18.75 steps
Epoch 1/100
  16  I am offset of Generator index  0
  16  I am offset of Generator index  0
  16  I am offset of Generator index  0
  32  I am offset of Generator index  0
  32  I am offset of Generator index  0
  32  I am offset of Generator index  0
  48  I am offset of Generator index  0
  48  I am offset of Generator index  0
  48  I am offset of Generator index  0
 1/18 [>.............................] - ETA: 2:57 - loss: 10.8665  64  I am offset of Generator index  0
  64  I am offset of Generator index  0
  64  I am offset of Generator index  0
12/18 [==================>...........] - ETA: 11s - loss: 10.2065  128  I am offset of Generator index  0
  128  I am offset of Generator index  0
13/18 [===================>..........] - ETA: 9s - loss: 10.1898   144  I am offset of Generator index  0
  144  I am offset of Generator index  0
14/18 [=====================>........] 

Process Keras_worker_ForkPoolWorker-3:
Process Keras_worker_ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
Process Keras_worker_ForkPoolWorker-2:
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__(

 5/18 [=======>......................] - ETA: 12s - loss: 6.5261
Epoch 00002: loss improved from 9.45942 to 6.52323, saving model to /home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/caption_model.h5


Exception in thread Thread-11:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 463, in _handle_results
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 251, in recv
    return _ForkingPickler.loads(buf.getbuffer())
_pickle.UnpicklingError: invalid load key, '\xbb'.




Epoch 00002: saving model to /home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/caption_weights.h5
 5/18 [=======>......................] - ETA: 17s - loss: 6.5261

KeyboardInterrupt: 

Process Keras_worker_ForkPoolWorker-4:
Process Keras_worker_ForkPoolWorker-6:
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-5:
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 347, in put
    self._writer.send_bytes(obj)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt

In [107]:
import os
os.getcwd()

'/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model'